### Hoemwork 2: Testing Imputing Strategies

It accompanies Chapter 4 of the book.

Data for this exercise were kindly provided by [Sascha Caron](https://www.nikhef.nl/~scaron/).

Copyright: Julieta Gruszko (2025)

License: [BSD-3-clause](https://opensource.org/license/bsd-3-clause/)




In [1]:
import numpy as np
import itertools
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.svm import SVC, LinearSVC # New algorithm!
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict, cross_validate
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 100)
rc('text', usetex=False)

We'll work with the 5000-event subset we saved in Studio 4. I'll load it for you below.

In [ ]:
features = pd.read_csv('../Data/ParticleID_features.csv', index_col='ID')

In [15]:
features.head()

,MET,METphi,P0_type,P0_E,P0_pt,P0_eta,P0_phi,P1_type,P1_E,P1_pt,P1_eta,P1_phi,P2_type,P2_E,P2_pt,P2_eta,P2_phi,P3_type,P3_E,P3_pt,P3_eta,P3_phi,P4_type,P4_E,P4_pt,P4_eta,P4_phi,P5_type,P5_E,P5_pt,P5_eta,P5_phi,P6_type,P6_E,P6_pt,P6_eta,P6_phi,P7_type,P7_E,P7_pt,P7_eta,P7_phi,P8_type,P8_E,P8_pt,P8_eta,P8_phi,P9_type,P9_E,P9_pt,P9_eta,P9_phi,P10_type,P10_E,P10_pt,P10_eta,P10_phi,P11_type,P11_E,P11_pt,P11_eta,P11_phi,P12_type,P12_E,P12_pt,P12_eta,P12_phi,P13_type,P13_E,P13_pt,P13_eta,P13_phi,P14_type,P14_E,P14_pt,P14_eta,P14_phi,P15_type,P15_E,P15_pt,P15_eta,P15_phi,P16_type,P16_E,P16_pt,P16_eta,P16_phi,P17_type,P17_E,P17_pt,P17_eta,P17_phi
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,62803.5,-1.810010,j,137571.0,128444.0,-0.345744,-0.307112,j,174209.0,127932.0,0.826569,2.332000,b,86788.9,84554.9,-0.180795,2.187970,j,140289.0,76955.8,-1.19933,-1.302800,m+,85230.6,70102.4,-0.645689,-1.659540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,57594.2,-0.509253,j,161529.0,80458.3,-1.318010,1.402050,j,291490.0,68462.9,-2.126740,-2.582310,e-,44270.1,35139.6,-0.706120,-0.371392,e+,72883.9,26902.2,-1.65386,-3.129630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,82313.3,1.686840,b,167130.0,113078.0,0.937258,-2.068680,j,102423.0,54922.3,1.226850,0.646589,j,60768.9,36244.3,1.102890,-1.434480,j,77714.0,27801.5,1.68461,1.389690,j,26840.0,24469.3,-0.388937,-1.647260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30610.8,2.617120,j,112267.0,61383.9,-1.211050,-1.457800,b,40647.8,39472.0,-0.024646,-2.222800,j,201589.0,32978.6,-2.496040,1.137810,j,90096.7,26964.5,1.87132,0.817631,j,28235.4,25887.9,-0.411528,2.024290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,45153.1,-2.241350,j,178174.0,100164.0,1.166880,-0.018721,j,92351.3,69762.1,0.774114,2.568740,j,61625.2,50086.7,0.652572,-3.012800,j,104193.0,31151.0,1.87641,0.865381,j,746585.0,26219.3,4.041820,-0.874169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
features.shape

(5000, 92)

In [ ]:
y = np.genfromtxt('../Data/ParticleID_labels.txt', dtype = str)

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() #turns categorical into 1 ... N

y = le.fit_transform(y)

target = np.abs(y - 1) # flip the labels

target

#### Let's take a look at these features, using the "describe" property.

In [25]:
features.describe() #Note that this automatically excludes non-numerical type columns

,MET,METphi,P0_E,P0_pt,P0_eta,P0_phi,P1_E,P1_pt,P1_eta,P1_phi,P2_E,P2_pt,P2_eta,P2_phi,P3_E,P3_pt,P3_eta,P3_phi,P4_E,P4_pt,P4_eta,P4_phi,P5_E,P5_pt,P5_eta,P5_phi,P6_E,P6_pt,P6_eta,P6_phi,P7_E,P7_pt,P7_eta,P7_phi,P8_E,P8_pt,P8_eta,P8_phi,P9_E,P9_pt,P9_eta,P9_phi,P10_E,P10_pt,P10_eta,P10_phi,P11_E,P11_pt,P11_eta,P11_phi,P12_E,P12_pt,P12_eta,P12_phi,P13_E,P13_pt,P13_eta,P13_phi,P14_E,P14_pt,P14_eta,P14_phi,P15_type,P15_E,P15_pt,P15_eta,P15_phi,P16_type,P16_E,P16_pt,P16_eta,P16_phi,P17_type,P17_E,P17_pt,P17_eta,P17_phi
count,5000.000000,5000.000000,5.000000e+03,5.000000e+03,5000.000000,5000.000000,4.997000e+03,4.997000e+03,4997.000000,4997.000000,4.950000e+03,4950.000000,4950.000000,4950.000000,4.717000e+03,4717.000000,4717.000000,4717.000000,4.002000e+03,4002.000000,4002.000000,4002.000000,2.871000e+03,2871.00000,2871.000000,2871.000000,1.889000e+03,1889.000000,1889.000000,1889.000000,1.186000e+03,1186.000000,1186.000000,1186.000000,7.290000e+02,729.000000,729.000000,729.000000,4.420000e+02,442.000000,442.000000,442.000000,2.610000e+02,261.000000,261.000000,261.000000,1.270000e+02,127.000000,127.000000,127.000000,5.600000e+01,56.000000,56.000000,56.000000,14.000000,14.000000,14.000000,14.000000,3.000000,3.000000,3.000000,3.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,64071.074332,-0.028916,3.301357e+05,1.540486e+05,-0.039812,-0.003049,2.527799e+05,1.080302e+05,-0.029936,0.007327,2.117980e+05,74863.343131,-0.025104,0.011845,1.805997e+05,57289.049481,0.010723,0.045266,1.780366e+05,48798.018516,0.015167,-0.031312,1.705620e+05,44042.67015,-0.022948,0.014522,1.628825e+05,41151.069666,0.002228,0.006738,1.581409e+05,40250.387015,0.072349,-0.035907,1.596814e+05,40139.289849,0.061654,-0.045868,1.574039e+05,39703.038235,0.118543,0.024249,1.561160e+05,38173.716092,0.029455,0.026422,1.631051e+05,34876.849606,0.206978,-0.001085,1.456600e+05,36151.183929,-0.000879,0.219260,180401.885714,27076.621429,-0.276634,0.595697,205102.000000,24610.900000,-1.333743,1.518666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,60525.122480,1.819257,3.068202e+05,1.149469e+05,1.361762,1.814855,2.638580e+05,8.136261e+04,1.439105,1.828832,2.510361e+05,46309.512365,1.577316,1.802715,2.383403e+05,32013.857623,1.634072,1.812078,2.577958e+05,26252.978520,1.744489,1.784248,2.381745e+05,23510.65367,1.806611,1.811101,2.269341e+05,20988.953157,1.815312,1.771888,2.118782e+05,26556.025657,1.836492,1.796932,2.308620e+05,30074.756789,1.842798,1.788596,2.165489e+05,30502.312276,1.872084,1.826435,2.319016e+05,29324.658352,1.884750,1.753017,2.248603e+05,20433.767238,1.998859,1.949004,1.943657e+05,25861.883410,1.941707,1.910400,262972.929941,3332.496623,2.181392,1.760349,230084.978534,1916.495834,2.661912,1.672151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,290.756000,-3.141010,3.857940e+04,2.825400e+04,-4.110220,-3.140710,1.087540e+04,1.080000e+04,-4.668790,-3.140530,1.221050e+04,10639.800000,-4.520250,-3.141480,1.169190e+04,10818.000000,-4.616550,-3.136130,1.110310e+04,10287.000000,-4.778980,-3.139040,1.070330e+04,10066.90000,-4.930230,-3.140380,1.197700e+04,11260.200000,-4.758150,-3.135630,1.380860e+04,10973.300000,-4.606330,-3.132610,1.119760e+04,10067.900000,-4.814380,-3.136380,1.615530e+04,10183.700000,-4.803880,-3.135910,2.004750e+04,14800.200000,-4.400470,-3.130690,1.780380e+04,12987.900000,-4.447660,-3.139820,2.512510e+04,14836.000000,-4.448760,-2.990730,25937.400000,23170.800000,-4.072000,-2.709890,58941.000000,23025.500000,-3.709270,-0.201051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,24352.375000,-1.619905,1.369522e+05,8.883690e+04,-1.035570,-1.574213,1.007510e+05,6.321840e+04,-1.060500,-1.602460,7.636905e+04,46549.475000,-1.125620,-1.547418,5.999090e+04,36097.700000,-1.121240,-1.518030,5.278370e+04,30891.650000,-1.198468,-1.550615,5.007050e+04,28453.95000,-1.250050,-1.586675,4.695560e+04,27963.500000,-1.231420,-1.475380,4.535515e+04,27140.550000,-1.243962,-1.626688,4.387110e+04,26825.

#### Imputing: mean values

In studio, we just worked with the track information for the first 4 products, and filled with 0's as our imputing strategy. 

This time, we'll use more of the tracks and try a slightly better imputing strategy:
- To avoid meaningless features, use all the tracks that occur it at least 1/5th of the events in your learning set. 
- Use sk-learn's $\texttt{SimpleImputer}$ to fill missing values with the mean for the feature. Make sure you think about how to prevent information leakage from the test set to the train set in this step!
- Train a scaled linear SVM and compare its performance to the benchmark model developed in Studio 4. 


In [ ]:
# your code here

Discuss the performancance of this model, comparing it to benchmark model in Studio 4. 

#### Imputing: kNN

Starting from the same list of features you used above, let's try something more sophisticated. ML models can be used for imputation instead, and sk-learn provides built-in methods for some common approaches. 

sk-learn provides the $\texttt{KNNImputer}$ class to use k-Nearest Neighbors as a technique to impute missing values. Each missing feature is imputed from the k Nearest Neighbors that have a value for that feature. See the documentation for more information!


Think back to what you know about kNN. Should you scale your data before or after imputing the missing values? Explain your reasoning. 

Avoiding information leakage between the test and train sets, scale the features and impute the missing values (using whatever order you advocated for above) and train another linear SVM model. Compare its performance to the benchmark model from Studio 4 and the simpler imputing version above. 

In [ ]:
# your code here

### Acknowledgement statement: